In [ ]:
import xarray as xr
import dask
import dask.array as da
import numpy as np
import scipy
from scipy import integrate
import h5py
import os
import subprocess

#raise the chunk size to better exploit the fact that seismo4 has tons of memory

from dask.distributed import Client, LocalCluster,progress,performance_report,wait
cluster = LocalCluster(n_workers=20) #,threads_per_worker=1
client =Client(cluster)
fullPath = os.getcwd()
print("localhost:"+str(client.scheduler_info()['services']['dashboard']))
dask.config.set({'array.chunk-size': '2048MiB'})

In [ ]:
#essiOutputs = os.listdir('.')
#essiOutputs = [i for i in essiOutputs if(".hdf5") in i][0]
#essiOutputs = "Location47faultNormalLarge.cycle=00000.essi"
essiOutputs = "fullDomain1Hz.cycle=0000.essi"
sw4Output = h5py.File(essiOutputs,'r')
print("timestep=" +str(sw4Output['timestep'][0]))
surfaceOnly = False
dtStep = 10 #I am dumping all of them so that this works better within paraview
#create a file name for the hdf5 paraview data
dispFname = sw4Output.filename.split('/')[-1].split('.')[0]+"DISP_PARAVIEW.hdf5"
rotFname = sw4Output.filename.split('/')[-1].split('.')[0]+"ROT_PARAVIEW.hdf5"
#name of the directory to save the xmf outputs
xmfOutputDir = "dispXMFout"
zipOutputs = True #zip the xmf output directory

In [ ]:
#paraviewData = h5py.File(paraviewFname,'a')
#get the dimensions
h=sw4Output["ESSI xyz grid spacing"][0]
#just get surface [:,:,:,:surfaceElevation]
Nx,Ny,Nz = sw4Output['vel_0 ijk layout'].shape[1],sw4Output['vel_0 ijk layout'].shape[2],sw4Output['vel_0 ijk layout'].shape[3]
xOrg,yOrg,zOrg = sw4Output['ESSI xyz origin'][:]
if surfaceOnly:
    surfaceElevation=2
    Nz=2
else: 
    surfaceElevation=None
timeSteps = sw4Output['vel_0 ijk layout'].shape[0]
#sw4 time step
dt = sw4Output['timestep'][:][0]

In [ ]:
# Data dumping step
#preprocess the hdf5 file makeing a fucking snap for each time step since paraview that poorly desinged and documented
"""
try:
    paraviewData.create_group("XparaviewSnaps")
    paraviewData.create_group("YparaviewSnaps")
    paraviewData.create_group("ZparaviewSnaps")

except ValueError:
    del paraviewData["XparaviewSnaps"]
    del paraviewData["YparaviewSnaps"]
    del paraviewData["ZparaviewSnaps"]
"""
#load up the origin
#load up the sw4 origin
sw4Xorigin,sw4Yorigin,sw4Zorigin = sw4Output["ESSI xyz origin"][:]
#close the paraview data so that I can access it directly from dask


In [ ]:
#integrate the dask data with scipy cumtrapz
def integrateDask(block,block_info=None,dt=dt):
    #print(block.shape)
    return integrate.cumtrapz(y=block,dx=dt,initial=0,axis=0)


xDisp = da.from_array(sw4Output['vel_0 ijk layout'])
yDisp = da.from_array(sw4Output['vel_1 ijk layout'])
zDisp = da.from_array(sw4Output['vel_2 ijk layout'])
#reformate array ranges to ditch data layers (if chosen)
xDisp = xDisp[:,:,:,:surfaceElevation]
yDisp = yDisp[:,:,:,:surfaceElevation]
zDisp = zDisp[:,:,:,:surfaceElevation]

print("lazy loaded complete domain")
print("instantiated dask arrays to correct range")
print("rechunked dask arrays")

#integrate
xDisp = da.map_blocks(integrateDask,xDisp,dtype=np.float32)
yDisp = da.map_blocks(integrateDask,yDisp,dtype=np.float32)
zDisp = da.map_blocks(integrateDask,zDisp,dtype=np.float32)
#flip z up and down to correct for difference in sw4 data
zDisp = da.flip(zDisp,axis=3)
print("lazy integrated")
### xarray dimensions
dims = ['dt','x','y','z']
coords = {'dt':np.arange(0,xDisp.shape[0],1),"x":np.arange(xOrg,xOrg+Nx*h,h),"y":np.arange(yOrg,yOrg+Ny*h,h),"z":np.arange(zOrg,zOrg+Nz*h,h)}
### now convert these to xarray objects
xDisp = client.persist(xr.DataArray(xDisp,dims=dims,coords=coords,name='xDisp'))
wait(xDisp)
xDisp.to_netcdf(dispFname,'w',group="xDisp",encoding={'xDisp':{"dtype": "float32",'zlib': True,'complevel': 9}})

del xDisp
print("saved x Disp")

yDisp = client.persist(xr.DataArray(yDisp,dims=dims,coords=coords,name='yDisp'))
yDisp.to_netcdf(dispFname,'a',group="yDisp",encoding={'yDisp':{"dtype": "float32",'zlib': True,'complevel': 9}})
wait(yDisp)
print("saved y Disp")
del yDisp

zDisp = client.persist(xr.DataArray(zDisp,dims=dims,coords=coords,name='zDisp'))
wait(zDisp)
zDisp.to_netcdf(dispFname,'a',group="zDisp",encoding={'zDisp':{"dtype": "float32",'zlib': True,'complevel': 9}})
print("saved z Disp")
del zDisp

"""
#save integrated results to the hdf5 file
da.to_hdf5(paraviewFname,'/xDisp',xDisp)
print("integrated and saved xDisp")
da.to_hdf5(paraviewFname,'/yDisp',yDisp)
print("integrated and saved yDisp")
da.to_hdf5(paraviewFname,'/zDisp',zDisp)
print("integrated and saved zDisp")
"""

In [ ]:
#close the sw4 output
sw4Output.close()

In [ ]:
#this is very slow if accessed as an xarray
#xDisp,yDisp,zDisp = xr.open_dataarray(dispFname,group="xDisp"),xr.open_dataset(dispFname,group="yDisp"),xr.open_dataset(dispFname,group="zDisp")
dispData = h5py.File(dispFname,'r')
xData,yData,zData = dispData['xDisp']['xDisp'][::dtStep,:,:,:],dispData['yDisp']['yDisp'][::dtStep,:,:,:],dispData['zDisp']['zDisp'][::dtStep,:,:,:]
dispData.close()


In [ ]:
#make the output directory for the individual hdf5 frames
os.makedirs(xmfOutputDir,exist_ok=True)
paraviewFnames = []
#loop through each dask data set and save the results
for t in range(xData.shape[0]):
    #name the output data set
    #what is this datas actual timestep?
    dtValue = t*dtStep
    displacementFilename = dispFname.split('_')[0]+"dt="+str(dtValue)+dispFname.split('_')[1]
    #create the output data set
    xmfData = h5py.File(displacementFilename,'w')
    #reshape these values to confrom to paraview coordinate shapes
    zDispParaview = np.flip(zData[t,:,:,:],axis=0)  
    #shape =x.shape
    #x=x.reshape(x.shape[-1],x.shape[1],x.shape[0])
    #y=y.reshape(y.shape[-1],y.shape[1],y.shape[0])
    #zDispParaview = zDispParaview.reshape(zDispParaview.shape[-1],zDispParaview.shape[1],zDispParaview.shape[0])
    xmfData.create_dataset('XparaviewSnaps', data=xData[t,:,:,:])
    xmfData.create_dataset('YparaviewSnaps', data=yData[t,:,:,:])
    xmfData.create_dataset('ZparaviewSnaps', data=zDispParaview)
    paraviewFnames.append(xmfData.filename)
    xmfData.close()
    #use subprocess so that I dont have to wait on this
    subprocess.Popen("mv " + displacementFilename + " ./"+xmfOutputDir+"/"+displacementFilename,shell=True)
    


In [ ]:
#now save the output data to an xmf file
### UPDATE THIS SO IT PULLS THIS INFORMATION FROM HDF5
print("HardCoding data type")
print("SPACING IS SET TO 1")
#dtype = sw4Output["vel_0 ijk layout"].dtype
dtype="Float"
precision="4"

f = open(dispFname.split('.')[0]+".xmf", 'w')
# Header for xml file
f.write('''<?xml version="1.0" ?>
<!DOCTYPE Xdmf SYSTEM "Xdmf.dtd" []>
<Xdmf Version="2.0">
<Domain>
<Grid Name="Box" GridType="Collection" CollectionType="Temporal">
''')

# loop over the attributes name written using time

t = 0
for files in range(len(paraviewFnames)):
    # Naming datasets 
    dataSetName1 = paraviewFnames[files]+":/XparaviewSnaps/"
    dataSetName2 = paraviewFnames[files]+":/YparaviewSnaps/"
    dataSetName3 = paraviewFnames[files]+":/ZparaviewSnaps/"
    # at individual time write the time independent Box grid. is it overdoing?
    
    f.write('''
    <!-- time step -->
    <Grid Name="Box %d" GridType="Uniform">  
    <Topology TopologyType="3DCoRectMesh" Dimensions="%d %d %d"/>
    <Geometry GeometryType="ORIGIN_DXDYDZ">
       <DataItem DataType="Float" Dimensions="3" Format="XML">0.0 0.0 0.0</DataItem>
       <DataItem DataType="Float" Dimensions="3" Format="XML">%d %d %d</DataItem>
    </Geometry>
    <Time Value="%d" />
    \n'''%(t, Nx, Ny, Nz,h,h,h, t))

    #write z first, paraview doesnt work if i dont for unkown reasons!

    f.write('''\n
    <Attribute Name="Z" AttributeType="Scalar" Center="Node">
    <DataItem Dimensions="%d %d %d" NumberType="Float" Precision="4"
    Format="HDF">
    %s
    </DataItem>
    </Attribute>
    \n'''%(Nx, Ny, Nz, dataSetName3))

    # First Attribute
    f.write('''\n
    <Attribute Name="X" AttributeType="Scalar" Center="Node">
    <DataItem Dimensions="%d %d %d" NumberType="Float" Precision="4"
    Format="HDF">
    %s
    </DataItem>
    </Attribute>
    \n'''%(Nx, Ny, Nz, dataSetName1))


    # Second Attribute
    f.write('''\n
    <Attribute Name="Y" AttributeType="Scalar" Center="Node">
    <DataItem Dimensions="%d %d %d " NumberType="Float" Precision="4"
    Format="HDF">
    %s
    </DataItem>
    </Attribute>
    </Grid>\n'''%(Nx, Ny, Nz, dataSetName2))
    # Third Attribute
    f.write('''\n
    <Attribute Name="Z" AttributeType="Scalar" Center="Node">
    <DataItem Dimensions="%d %d %d" NumberType="Float" Precision="4"
    Format="HDF">
    %s
    </DataItem>
    </Attribute>
    \n'''%(Nx, Ny, Nz, dataSetName3))



    ### for some reason paraview wont read my last Z attribute unless there is a placehold
    f.write('''\n
    <Attribute Name="ZBlank" AttributeType="Scalar" Center="Node">
    <DataItem Dimensions="%d %d %d" NumberType="Float" Precision="4"
    Format="HDF">
    %s
    </DataItem>
    </Attribute>
    \n'''%(Nz, Ny, Nx, dataSetName3))
    
    ### update timestep t
    #print(t)
    t += dtStep
    


# End the xmf file
f.write('''
   </Grid>
</Domain>
</Xdmf>
''')
f.flush()
f.close()

#move thie file to the output directory
subprocess.Popen("mv " + dispFname.split('.')[0]+".xmf" + " ./"+xmfOutputDir+"/"+dispFname.split('.')[0]+".xmf",shell=True)



In [ ]:
### if requested compress the output
if(zipOutputs): subprocess.Popen("zip -r " + xmfOutputDir +".zip " + xmfOutputDir,shell=True)
